<a href="https://colab.research.google.com/github/silverstar0727/Kaggle_Santander-Product-Recommendation/blob/master/santander.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 목표
고객이 신규로 구매할 제품이 무엇인가?

-> 고객이 2016-05-28 시점에 보유하고 있지 않은 금융 제품 중에서 2016-06-28에 구매할 것으로 예측되는 제품 상위 7개를 제출해야 한다.

## 평가척도 (MAP@7)
MAP은 Mean Average Precision이다. 먼저 Average Precision은 예측 정확도의 평균을 의미한다.
> 예를 들어 7개의 칼럼에 대한 예측을 정답은 1, 오답은 0이라고 할 때, 칼럼에 대한 예측결과가 1001110이라면, Average Precision은 1/1, 0, 0, 2/4, 4/6, 0이다.

Mean Average Precision은 모든 예측 결과물의 Average Precision의 평균값을 의미하고 @7이 붙는 이유는 최대 7개의 칼럼을 예측할 수 있다는 것을 의미한다.
> 위 예제에서는 (1/1 + 2/4 + 3/5 + 4/6) / 4 = 0.69 였다.

이러한 MAP@7 평가척도의 과정을 보면 예측의 순서에 매우 예민하다는 것을 알 수 있다.
> 7개의 예측 결과물 중 4개의 정답이 모두 처음에 위치하여 1111000과 같다면 Mean Average Precision은 (1/1 + 2/2 + 3/3 + 4/4) / 4 = 1이므로 최고 점수인 1이 될 것이다. 반대로 0001111이라면 Mean Average Precision은 0.43이 된다.

경진대회에서 MAP@7 평가 척도를 구하기 위해서는 다음의 코드를 사용한다. mapk()의 입력 값으로 들어가는 actual, predicted는 고객의 수 * 7의 dimension을 갖는 list of list이다.

In [ ]:
import numpy as np

def apk(acual, predicted, k = 7, default = 0.0):
  # MAP@7 이므로 최대 7개만 사용한다.
  if len(predicted) > k:
    predicted = predicted[:k]

  score = 0.0
  num_hits = 0.0

  for i, p in enumerate(predicted):
    # 점수를 부여하는 조건은 다음과 같다.
    # 예측 값이 정답에 있고 ('p in actual')
    # 예측값이 중복이 아니면 ('p not in predicted[:i]')
    if (p in actual) & (p not in predicted[:i]):
      num_hits += 1.0
      score += num_hits / (i + 1.0)

  # 정답값이 공백일 경우, 무조건 0.0점을 반환함
  if not actual:
    return default

  # 정답의 개수(len(actual))로 average precision을 구한다
  return score / min(len(actual), k)

def mapk(acual, predicted, k = 7, default = 0.0):
  # list of list인 정답 값(actual)과 예측값(predicted)에서 고객별 Average Precision을 구하고 np.mean()을 통해
  # 평균을 계산함
  return np.mean([apk(a, p, k, default) for a, p, in zip(actual, predicted)])

이번 경진대회에서는 Tabular형태의 시계열 데이터를 다루는데, 이 경우에는 딥러닝 모델보다 트리 기반의 앙상블 모델이 더 좋은 성능을 지닌다.

여기서는 주로 XGBoost와 LightGBM을 사용한다.

XGBoost 홈페이지: http://xgboost/readthedocs.io/en/latest

LightGBM: https://papers/nips.cc/paper/6907-lightgbm-a-highly-efficient-gradient-boosting-tree

In [ ]:
import pandas as pd
import numpy as np

train = pd.read_csv('/content/drive/My Drive/santander-product-recommendation/train_ver2.csv')

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (5,8,11,15) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [ ]:
# shape함수를 통한 훈련데이터의 크기 확인
train.shape

(13647309, 48)

In [ ]:
# 첫 다섯줄을 통해 데이터를 직접확인
train.head()

,fecha_dato,ncodpers,ind_empleado,pais_residencia,sexo,age,fecha_alta,ind_nuevo,antiguedad,indrel,ult_fec_cli_1t,indrel_1mes,tiprel_1mes,indresi,indext,conyuemp,canal_entrada,indfall,tipodom,cod_prov,nomprov,ind_actividad_cliente,renta,segmento,ind_ahor_fin_ult1,ind_aval_fin_ult1,ind_cco_fin_ult1,ind_cder_fin_ult1,ind_cno_fin_ult1,ind_ctju_fin_ult1,ind_ctma_fin_ult1,ind_ctop_fin_ult1,ind_ctpp_fin_ult1,ind_deco_fin_ult1,ind_deme_fin_ult1,ind_dela_fin_ult1,ind_ecue_fin_ult1,ind_fond_fin_ult1,ind_hip_fin_ult1,ind_plan_fin_ult1,ind_pres_fin_ult1,ind_reca_fin_ult1,ind_tjcr_fin_ult1,ind_valo_fin_ult1,ind_viv_fin_ult1,ind_nomina_ult1,ind_nom_pens_ult1,ind_recibo_ult1
0,2015-01-28,1375586,N,ES,H,35,2015-01-12,0.0,6,1.0,NaN,1,A,S,N,NaN,KHL,N,1.0,29.0,MALAGA,1.0,87218.10,02 - PARTICULARES,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0
1,2015-01-28,1050611,N,ES,V,23,2012-08-10,0.0,35,1.0,NaN,1,I,S,S,NaN,KHE,N,1.0,13.0,CIUDAD REAL,0.0,35548.74,03 - UNIVERSITARIO,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0
2,2015-01-28,1050612,N,ES,V,23,2012-08-10,0.0,35,1.0,NaN,1,I,S,N,NaN,KHE,N,1.0,13.0,CIUDAD REAL,0.0,122179.11,03 - UNIVERSITARIO,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0
3,2015-01-28,1050613,N,ES,H,22,2012-08-10,0.0,35,1.0,NaN,1,I,S,N,NaN,KHD,N,1.0,50.0,ZARAGOZA,0.0,119775.54,03 - UNIVERSITARIO,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0
4,2015-01-28,1050614,N,ES,V,23,2012-08-10,0.0,35,1.0,NaN,1,A,S,N,NaN,KHE,N,1.0,50.0,ZARAGOZA,1.0,NaN,03 - UNIVERSITARIO,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0


다양한 유형의 변수가 존재하는 것을 확인할 수 있음

for문을 통해서 모든 변수의 첫 5줄을 미리보기

In [ ]:
for col in train.columns:
  print('{')